# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [1]:
import requests
root_url = "https://country-leaders.onrender.com" 
status_url = "status"
req = requests.get(f"{root_url}/{status_url}")

if req.status_code == 200:    
    print(req.text) 
else:
    print(f"Error to access the website: {req.status_code}") 

"Alive"


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [2]:
countries_url = "countries"
req = requests.get(f"{root_url}/{countries_url}")
countries = req.json()
print(countries)


{'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [3]:
cookie_url = "cookie"    
cookie = requests.get(f"{root_url}/{cookie_url}").cookies
print(cookie)

<RequestsCookieJar[<Cookie user_cookie=92bb6a73-bd33-48bf-9f68-5f7c2667dfd0 for country-leaders.onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [4]:
r = requests.get(f"{root_url}/{countries_url}", cookies=cookie)
countries = r.json()
print(countries)

['us', 'be', 'ma', 'ru', 'fr']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [5]:
leaders_url = 'leaders'
leaders = requests.get(f"{root_url}/{leaders_url}", cookies=cookie) 
print(leaders)

<Response [404]>


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [6]:
    
leader = requests.get(f"{root_url}/{leaders_url}", cookies=cookie, params={"country":countries}) 
# print (leader.json())

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [7]:
leaders_per_country = {}
for country in countries:        
        leader = requests.get(f"{root_url}/{leaders_url}", cookies=cookie, params={"country":country})        
        leaders_per_country[country] = leader.json()
# print (leaders_per_country)


In [8]:
# or 1 line


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [9]:
import requests
def get_leaders():
    root_url = "https://country-leaders.onrender.com" 
    status_url = "status"
    req = requests.get(f"{root_url}/{status_url}")
    if req.status_code == 200:    
        print(req.text) 
    else:
        print(f"Error to access the website: {req.status_code}")       
    
    countries_url = "countries"
    cookie_url = "cookie"    
    cookie = requests.get(f"{root_url}/{cookie_url}").cookies
    r = requests.get(f"{root_url}/{countries_url}", cookies=cookie)
    countries = r.json()   
   
    leaders_url = 'leaders'
    leaders_per_country = {}
    for country in countries:        
            leader = requests.get(f"{root_url}/{leaders_url}", cookies=cookie, params={"country":country})        
            leaders_per_country[country] = leader.json()
    return leaders_per_country



Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [10]:
# 2 lines
leaders_per_country = get_leaders()

"Alive"


## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [11]:
first_leader = next(iter(leaders_per_country.values()))[0]
wiki_url = first_leader.get("wikipedia_url")
r = requests.get(wiki_url)
# print(r.text)


Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [12]:
# 3 lines


That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [13]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')
paragraphs = soup.find_all('p')
# print(paragraphs)

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [14]:
first_paragraph = None
for paragraph in paragraphs:
    text = paragraph.get_text(strip=True)
    if text:  # Check if the paragraph has non-empty text
        first_paragraph = text
        break
print(first_paragraph)

George Washington(February 22, 1732 –December 14, 1799) was aFounding Father of the United States, military officer, and farmer who served as the firstpresident of the United Statesfrom 1789 to 1797. Appointed by theSecond Continental Congressas commander of theContinental Armyin 1775, Washington ledPatriotforces to victory in theAmerican Revolutionary Warand then served as president of theConstitutional Conventionin 1787, which drafted the currentConstitution of the United States. Washington has thus become commonly known as the "Father of his Country".


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [15]:
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)        
    r = requests.get(wikipedia_url)   
    soup = BeautifulSoup(r.text, 'html.parser')
    paragraphs = soup.find_all('p')    
    first_paragraph = None
    for paragraph in paragraphs:
        text = paragraph.get_text(strip=True)
        if text:  #check non-empty paragraphs
            first_paragraph = text
            break    
    return first_paragraph

In [16]:
# Test: 3 lines
first_paragraph = get_first_paragraph(wiki_url)
print(first_paragraph)

https://en.wikipedia.org/wiki/George_Washington
George Washington(February 22, 1732 –December 14, 1799) was aFounding Father of the United States, military officer, and farmer who served as the firstpresident of the United Statesfrom 1789 to 1797. Appointed by theSecond Continental Congressas commander of theContinental Armyin 1775, Washington ledPatriotforces to victory in theAmerican Revolutionary Warand then served as president of theConstitutional Conventionin 1787, which drafted the currentConstitution of the United States. Washington has thus become commonly known as the "Father of his Country".


### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [17]:
import re    
first_paragraph = re.sub(r'\s+', ' ', first_paragraph)    
first_paragraph = re.sub(r'([a-zA-Z])\s?\(', r'\1(', first_paragraph)
first_paragraph = re.sub(r'\.([A-Za-z])', r'. \1', first_paragraph)
first_paragraph = re.sub(r'([a-z])([A-Z])', r'\1 \2', first_paragraph)

print(first_paragraph)



George Washington(February 22, 1732 –December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the firstpresident of the United Statesfrom 1789 to 1797. Appointed by the Second Continental Congressas commander of the Continental Armyin 1775, Washington led Patriotforces to victory in the American Revolutionary Warand then served as president of the Constitutional Conventionin 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".


Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [18]:
# 10 lines

def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)   
    r = requests.get(wikipedia_url)    
    soup = BeautifulSoup(r.text, 'html.parser')
    paragraphs = soup.find_all('p')

    first_paragraph = None
    for paragraph in paragraphs:
        text = paragraph.get_text(strip=True)
        if text:  #check non-empty paragraphs
            first_paragraph = text
            break

    first_paragraph = re.sub(r'\s+', ' ', first_paragraph)    
    first_paragraph = re.sub(r'([a-zA-Z])\s?\(', r'\1(', first_paragraph)
    first_paragraph = re.sub(r'\.([A-Za-z])', r'. \1', first_paragraph)
    first_paragraph = re.sub(r'([a-z])([A-Z])', r'\1 \2', first_paragraph)
    return first_paragraph

# Test:
first_paragraph = get_first_paragraph(wiki_url)
print(first_paragraph)



https://en.wikipedia.org/wiki/George_Washington
George Washington(February 22, 1732 –December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the firstpresident of the United Statesfrom 1789 to 1797. Appointed by the Second Continental Congressas commander of the Continental Armyin 1775, Washington led Patriotforces to victory in the American Revolutionary Warand then served as president of the Constitutional Conventionin 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [19]:
# < 20 lines


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [20]:

def get_leaders():
    root_url = "https://country-leaders.onrender.com" 
    status_url = "status"
    req = requests.get(f"{root_url}/{status_url}")
    if req.status_code == 200:    
        print(req.text) 
    else:
        print(f"Error to access the website: {req.status_code}")        
    #get the cookie and countries
    countries_url = "countries"
    cookie_url = "cookie"    
    cookie = requests.get(f"{root_url}/{cookie_url}").cookies
    r = requests.get(f"{root_url}/{countries_url}", cookies=cookie)
    countries = r.json()    
    # leaders
    leaders_url = 'leaders'
    leaders_per_country = {}
    for country in countries:        
            leader_r = requests.get(f"{root_url}/{leaders_url}", cookies=cookie, params={"country":country})        
            if leader_r.status_code == 200:
                leaders = leader_r.json()            
            leaders_per_country[country] = leaders
            for leader in leaders:
                print(f"Leader of {country}: {leader.get('first_name')} {leader.get('last_name')}, Wikipedia URL: {leader.get('wikipedia_url')}")
         
    return leaders_per_country

# Test:
leaders_data = get_leaders()


"Alive"
Leader of us: George Washington, Wikipedia URL: https://en.wikipedia.org/wiki/George_Washington
Leader of us: Barack Obama, Wikipedia URL: https://en.wikipedia.org/wiki/Barack_Obama
Leader of us: Abraham Lincoln, Wikipedia URL: https://en.wikipedia.org/wiki/Abraham_Lincoln
Leader of us: George Bush, Wikipedia URL: https://en.wikipedia.org/wiki/George_W._Bush
Leader of us: Bill Clinton, Wikipedia URL: https://en.wikipedia.org/wiki/Bill_Clinton
Leader of us: Joe Biden, Wikipedia URL: https://en.wikipedia.org/wiki/Joe_Biden
Leader of us: Franklin Roosevelt, Wikipedia URL: https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
Leader of us: Andrew Johnson, Wikipedia URL: https://en.wikipedia.org/wiki/Andrew_Johnson
Leader of us: Gerald Ford, Wikipedia URL: https://en.wikipedia.org/wiki/Gerald_Ford
Leader of us: Richard Nixon, Wikipedia URL: https://en.wikipedia.org/wiki/Richard_Nixon
Leader of us: Lyndon Johnson, Wikipedia URL: https://en.wikipedia.org/wiki/Lyndon_B._Johnson
Leader of

In [21]:
# Check the output of your function (2 lines)


Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [22]:
def get_leaders():
    root_url = "https://country-leaders.onrender.com" 
    status_url = "status"
    req = requests.get(f"{root_url}/{status_url}")
    if req.status_code == 200:    
        print(req.text) 
    else:
        print(f"Error to access the website: {req.status_code}")        
    #get the cookie and countries
    countries_url = "countries"
    cookie_url = "cookie"    
    cookie = requests.get(f"{root_url}/{cookie_url}").cookies
    r = requests.get(f"{root_url}/{countries_url}", cookies=cookie)
    countries = r.json()    
    # leaders
    leaders_url = 'leaders'
    leaders_per_country = {}
    for country in countries:        
            leader_r = requests.get(f"{root_url}/{leaders_url}", cookies=cookie, params={"country":country})        
            if leader_r.status_code == 200:
                leaders = leader_r.json() 
                country_leaders = {}
                for leader in leaders:
                    full_name = f"{leader.get('first_name')} {leader.get('last_name')}"
                    wikipedia_url = leader.get('wikipedia_url')
                    if wikipedia_url:                        
                        first_paragraph = get_first_paragraph(wikipedia_url)
                        country_leaders[full_name] = first_paragraph
                        print(f"Leader of {country}: {full_name}, First Paragraph: {first_paragraph}")
            leaders_per_country[country] = country_leaders         
    return leaders_per_country


def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)    
    r = requests.get(wikipedia_url)     
    # all paragraphs
    soup = BeautifulSoup(r.text, 'html.parser')
    paragraphs = soup.find_all('p')
    #get the 1st paragraph
    first_paragraph = None
    for paragraph in paragraphs:
        text = paragraph.get_text(strip=True)
        if text:  #check non-empty paragraphs
            first_paragraph = text
            break
        # clean up the text
    first_paragraph = re.sub(r'\s+', ' ', first_paragraph)    
    first_paragraph = re.sub(r'([a-zA-Z])\s?\(', r'\1(', first_paragraph)
    first_paragraph = re.sub(r'\.([A-Za-z])', r'. \1', first_paragraph)
    first_paragraph = re.sub(r'([a-z])([A-Z])', r'\1 \2', first_paragraph)
    
    return first_paragraph


Check the output of your function again.

In [23]:
# Check the output of your function (1 line)
leaders_data = get_leaders()


"Alive"
https://en.wikipedia.org/wiki/George_Washington
Leader of us: George Washington, First Paragraph: George Washington(February 22, 1732 –December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the firstpresident of the United Statesfrom 1789 to 1797. Appointed by the Second Continental Congressas commander of the Continental Armyin 1775, Washington led Patriotforces to victory in the American Revolutionary Warand then served as president of the Constitutional Conventionin 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".
https://en.wikipedia.org/wiki/Barack_Obama
Leader of us: Barack Obama, First Paragraph: Barack Hussein Obama II[a](born August 4, 1961) is an American politician who served as the 44thpresident of the United Statesfrom 2009 to 2017. As a member of the Democratic Party, he was the first African-Americanpresident in U. S. hist

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [24]:
# < 20 lines


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [25]:
from requests import Session
import time

def session_loop_request(session, url, cookies=None, params=None, retry=False):    
    response = session.get(url, cookies=cookies, params=params)
    if response.status_code == 403 and not retry:       #this fix the cookie expired problem  
        print("Cookie expired. Refreshing cookie and retrying...")
        new_cookie = refresh_cookie(session)
        return session_loop_request(session, url, cookies=new_cookie, params=params, retry=True)
    response.raise_for_status()
    if "json" in response.headers.get("Content-Type", ""):
        return response.json()
    return response.text

def refresh_cookie(session): #this fix the cookie expired problem    
    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    return session.get(cookie_url).cookies

def get_leaders():
    root_url = "https://country-leaders.onrender.com"    
    with Session() as session:        
        status_url = f"{root_url}/status"
        status_response = session.get(status_url)
        if status_response.status_code == 200:
            print(status_response.text)
        else:
            print(f"Error accessing the website: {status_response.status_code}")        
        cookie = refresh_cookie(session)
        countries_url = f"{root_url}/countries"
        countries = session_loop_request(session, countries_url, cookies=cookie)        
        leaders_per_country = {}
        leaders_url = f"{root_url}/leaders"
        for country in countries:
            country_leaders = {}
            leaders_data = session_loop_request(session, leaders_url, cookies=cookie, params={"country": country})            
            for leader in leaders_data:
                full_name = f"{leader.get('first_name')} {leader.get('last_name')}"
                wikipedia_url = leader.get('wikipedia_url')
                if wikipedia_url:                    
                    first_paragraph = get_first_paragraph(session, wikipedia_url)
                    country_leaders[full_name] = first_paragraph
                    print(f"Leader of {country}: {full_name}, First Paragraph: {first_paragraph}")            
            leaders_per_country[country] = country_leaders
            time.sleep(0.5)  # delay between countries to avoid overload
            
    return leaders_per_country

def get_first_paragraph(session, wikipedia_url):    
    print(wikipedia_url)
    response = session.get(wikipedia_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')
    first_paragraph = None
    for paragraph in paragraphs:
        text = paragraph.get_text(separator=" ",strip=True)# this fix also the wikilinks problem
        if text:
            first_paragraph = text
            break 

    
       
    return first_paragraph


Test your new functions.



In [26]:
leaders_data = get_leaders()

"Alive"
https://en.wikipedia.org/wiki/George_Washington
Leader of us: George Washington, First Paragraph: George Washington (February 22, 1732 – December 14 , 1799) was a Founding Father of the United States , military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States . Washington has thus become commonly known as the " Father of his Country ".
https://en.wikipedia.org/wiki/Barack_Obama
Leader of us: Barack Obama, First Paragraph: Barack Hussein Obama II [ a ] (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party , he was the first African-American p

## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [27]:
import json
leaders_data = get_leaders()
with open('leaders.json', 'w') as json_file:
    json.dump(leaders_data, json_file, indent=4)


"Alive"
https://en.wikipedia.org/wiki/George_Washington
Leader of us: George Washington, First Paragraph: George Washington (February 22, 1732 – December 14 , 1799) was a Founding Father of the United States , military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States . Washington has thus become commonly known as the " Father of his Country ".
https://en.wikipedia.org/wiki/Barack_Obama
Leader of us: Barack Obama, First Paragraph: Barack Hussein Obama II [ a ] (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party , he was the first African-American p

Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [28]:
with open('leaders.json', 'r') as json_file:
    data = json.load(json_file)
    print(data)

{'us': {'George Washington': 'George Washington (February 22, 1732\xa0– December 14 , 1799) was a Founding Father of the United States , military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States . Washington has thus become commonly known as the " Father of his Country ".', 'Barack Obama': 'Barack Hussein Obama II [ a ] (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party , he was the first African-American president in U.S. history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator f

Make a function `save(leaders_per_country)` to call this code easily.

In [29]:
def save(leaders_per_country):
    with open('leaders.json', 'w') as json_file:  
        json.dump(leaders_per_country, json_file, indent=4) 


In [30]:
save(leaders_data) 


## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!